# Olivin Example

Import the needed packages

In [ ]:
import phasefield_gps as gps
import ngsolve as ngs
from ngsolve.meshes import MakeStructured2DMesh
from ngsolve.webgui import Draw

We use ngsolve to create a structured quad-mesh of our simulation domain:

In [ ]:
mesh = MakeStructured2DMesh(nx=100, ny=10, mapping=lambda x,y: (1e-3*x-5e-4, 1e-4*y))
Draw(mesh)

Define phases, each phase needs a name and its diffusion coefficient:

In [ ]:
liquid = gps.Phase("liquid", diffusion_coefficient=3e-7)
solid = gps.Phase("solid", diffusion_coefficient=3e-12)

We model our components as ideal solutions, each component gets a name and phase energies for each of the phases we use:

In [ ]:
fosterite = gps.IdealSolutionComponent(name="fosterite",
                                       phase_energies={ liquid: -2498560,
                                                        solid: -2509411 })
fayalite = gps.IdealSolutionComponent(name="Fayalite",
                                      phase_energies={ liquid: -1980885,
                                                       solid: -1951843 })

Now we can create a `GrandPotentialSolver` to solve the phasefield equations on our mesh.

Note that the interface width is not the true physical one, but instead chosen to resolve structures of interest, but also be resolved by the mesh.

Using this model we can choose it larger than physically true without changing underlying physics, to be able to simulate it efficiently.

We can choose if the should enforce mass conservation, this will add one additional variable + equation, so make the solver slower. If we do not enforce mass conservation the total concentration over time will typically vary slightly.

In [ ]:
solver = gps.GrandPotentialSolver(mesh=mesh, components=[fosterite, fayalite],
                                  phases=[liquid, solid],
                                  molar_volume=4.3e-5,
                                  interface_mobility=1e-13,
                                  interface_energy=2.45,
                                  temperature=1923.15,
                                  interface_width=5e-5)
solver.mass_conservation = False

## Initial Conditions

We can set initial conditions for the phases.
Additionally we can set for each component for each phase an initial condition.

In [ ]:
tanh = lambda x: ngs.sinh(x)/ngs.cosh(x)
shift = 0
ic_concentration = { fayalite: { liquid: 0.999,
                                solid: 0.98 } }
ic_concentration[fosterite] = { liquid: 1-ic_concentration[fayalite][liquid],
                                solid: 1-ic_concentration[fayalite][solid] }
ic_liquid = 0.5 * tanh(ngs.sqrt(solver.m/solver.kappa) * (ngs.x-shift)) + 0.5
solver.set_initial_conditions({ liquid: ic_liquid,
                                solid: 1-ic_liquid },
                              components=ic_concentration)
scenes = []
scenes.append(Draw(solver.get_phase(liquid), mesh, height="200px"))
cs = solver.get_concentrations()
scenes.append(Draw(cs[fayalite], mesh, height="200px"))
scenes.append(Draw(cs[fosterite], mesh, height="200px"))

We can also compute some internal results, like the total concentration of fayalite:

In [ ]:
print(ngs.Integrate(cs[fayalite], mesh))

Let's set the timestep size and do one timestep:

In [ ]:
solver.dt.Set(0.1)
solver.do_timestep()

We can add a callback function to our solve routine, to output some values of interest in each timestep, and also update our scenes (for example all 10 timesteps)

In [ ]:
counter = 0
def callback():
    global counter
    print(f"Time: {solver.time}, energy: {ngs.Integrate(solver.get_energy(), mesh)}")
    counter += 1
    if counter % 10 == 0:
        for scene in scenes:
            scene.Redraw()

using the `solver` routine we can do multiple timesteps - calling the callback function after each successful.
We can change the timestep size anytime, let's try for example setting it to 1 and doing some timesteps.

We use the `TaskManager` so that computations run using NGSolve-parallelization

In [ ]:
with ngs.TaskManager():
    solver.dt.Set(1)
    solver.solve(20, callback=callback)

We see that the first timestep was not successive - 1 was to large. So the solver automatically reduced it and tried smaller timesteps until the newton method converged. We also see that our callback function was called outputting the time and energy of the system and in every 10th iteration updating the visualization.

We can now also compute the total mass of component 1:

In [ ]:
print(ngs.Integrate(cs[fayalite], mesh))

We see we lost a bit of fayalite in the simulation. Enable `solver.mass_conservation` to see the difference.